# XGBoost model with Airline dataset. Experiment 01

XGBoost regressor with logloss + Airline dataset

In [1]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from lightgbm.sklearn import LGBMRegressor
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import (confusion_matrix, accuracy_score, roc_auc_score, f1_score, log_loss, precision_score,
                             recall_score, mean_squared_error, mean_absolute_error, r2_score)
from scipy.stats import gmean

from libs.timer import Timer
from libs.loaders import load_fraud, load_iot, load_airline
from libs.conversion import convert_cols_categorical_to_numeric, convert_related_cols_categorical_to_numeric
from libs.conversion import _get_nominal_integer_dict, _convert_to_integer

print("System version: {}".format(sys.version))


System version: 3.5.2 |Anaconda custom (64-bit)| (default, Jul  2 2016, 17:53:06) 
[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]


### Load data

In [2]:
%%time
df_plane = load_airline()
print(df_plane.shape)


(115069017, 14)
CPU times: user 1min 31s, sys: 24.9 s, total: 1min 56s
Wall time: 4min 18s


In [3]:
df_plane.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,SFO,ORD,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,LAX,SFO,337,0,5
2,1987,10,1,4,5,35,HP,351,167,ICT,LAS,987,0,17
3,1987,10,1,4,5,40,DL,251,35,MCO,PBI,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,LAS,ORD,1515,0,17


In [4]:
%%time
df_plane_numeric = convert_related_cols_categorical_to_numeric(df_plane, col_list=['Origin','Dest'])


CPU times: user 1min 54s, sys: 13.7 s, total: 2min 8s
Wall time: 2min 12s


In [5]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,AA,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,EA,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,HP,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,DL,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,UA,500,208,4,33,1515,0,17


In [6]:
%%time
df_plane_numeric = convert_cols_categorical_to_numeric(df_plane_numeric, col_list='UniqueCarrier')


CPU times: user 1min 3s, sys: 10 s, total: 1min 13s
Wall time: 1min 16s


In [7]:
df_plane.dtypes

Year                  int64
Month                 int64
DayofMonth            int64
DayofWeek             int64
CRSDepTime            int64
CRSArrTime            int64
UniqueCarrier        object
FlightNum             int64
ActualElapsedTime     int64
Origin               object
Dest                 object
Distance              int64
Diverted              int64
ArrDelay              int64
dtype: object

In [8]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17


In [9]:
%%time
filter_func = lambda x: 1 if x > 0 else 0
df_plane_numeric['ArrDelayBinary'] = df_plane_numeric['ArrDelay'].map(filter_func)

CPU times: user 40.9 s, sys: 4.55 s, total: 45.4 s
Wall time: 46.3 s


In [10]:
df_plane_numeric.head()

,Year,Month,DayofMonth,DayofWeek,CRSDepTime,CRSArrTime,UniqueCarrier,FlightNum,ActualElapsedTime,Origin,Dest,Distance,Diverted,ArrDelay,ArrDelayBinary
0,1987,10,1,4,1,556,0,190,247,0,33,1846,0,27,1
1,1987,10,1,4,5,114,1,57,74,1,0,337,0,5,1
2,1987,10,1,4,5,35,2,351,167,2,4,987,0,17,1
3,1987,10,1,4,5,40,3,251,35,3,41,142,0,-2,0
4,1987,10,1,4,8,517,4,500,208,4,33,1515,0,17,1


In [11]:
%%time
#gets a random 80% of the entire set
X_train = df_plane_numeric.sample(frac=0.8, random_state=1)
print(X_train.shape)
#gets the left out portion of the dataset
X_test = df_plane_numeric.loc[~df_plane_numeric.index.isin(X_train.index)]
print(X_test.shape)

(92055214, 15)
(23013803, 15)
CPU times: user 1min 10s, sys: 15.6 s, total: 1min 26s
Wall time: 1min 29s


### Training

In [12]:
# Variables
KFOLD_SPLITS = 5

In [13]:
def train_xgboost(df, verbose=True):
    if verbose: print("Training data of size {}".format(df.shape))
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
    y = df['ArrDelayBinary'].as_matrix()
    
    n_splits = 5
    skf = KFold(n_splits=n_splits, random_state=77, shuffle=True)
    clfs = []
    
    if verbose: print("Computing boosted tree using {} kfold cross validation".format(n_splits))
    for train_index, test_index in skf.split(x, y):
        trn_x, val_x = x[train_index,:], x[test_index,:]
        trn_y, val_y = y[train_index], y[test_index] 
        
        clf = xgb.XGBRegressor(max_depth=8,
                              n_estimators=100,
                              min_child_weight=30,
                              learning_rate=0.1,
                              nthread=20,
                              subsample=0.80,
                              colsample_bytree=0.80,
                              seed=77)
        clf.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], verbose=verbose, eval_metric='logloss')
        clfs.append(clf)
        
    return clfs

In [14]:
%%time
X_train_small = X_train.sample(frac=0.001, random_state=1)
print(X_train_small.shape)

(92055, 15)
CPU times: user 4.84 s, sys: 268 ms, total: 5.1 s
Wall time: 5.28 s


In [15]:
%%time
#clfs = train_xgboost(X_train_small)
clfs = train_xgboost(X_train)

Training data of size (92055214, 15)
Computing boosted tree using 5 kfold cross validation
[0]	validation_0-logloss:0.680375
[1]	validation_0-logloss:0.669001
[2]	validation_0-logloss:0.665048
[3]	validation_0-logloss:0.661617
[4]	validation_0-logloss:0.658786
[5]	validation_0-logloss:0.656328
[6]	validation_0-logloss:0.654535
[7]	validation_0-logloss:0.645937
[8]	validation_0-logloss:0.644453
[9]	validation_0-logloss:0.642674
[10]	validation_0-logloss:0.641533
[11]	validation_0-logloss:0.640492
[12]	validation_0-logloss:0.639001
[13]	validation_0-logloss:0.632697
[14]	validation_0-logloss:0.626733
[15]	validation_0-logloss:0.621016
[16]	validation_0-logloss:0.61645
[17]	validation_0-logloss:0.612035
[18]	validation_0-logloss:0.608456
[19]	validation_0-logloss:0.60758
[20]	validation_0-logloss:0.603941
[21]	validation_0-logloss:0.60071
[22]	validation_0-logloss:0.598418
[23]	validation_0-logloss:0.595844
[24]	validation_0-logloss:0.593563
[25]	validation_0-logloss:0.592585
[26]	validat

### Evaluation

In [16]:
def compute_prediction(clfs, df, verbose=True):    
    x = df[df.columns.difference(['ArrDelay', 'ArrDelayBinary'])].as_matrix()
    if verbose: print(x.shape)
    preds = []
    with Timer() as t:
        for clf in clfs:
            preds.append(np.clip(clf.predict(x), 0.0001, 0.9999))
        pred = gmean(np.array(preds), axis=0)
    if verbose: print("Prediction took %.03f sec.\n" % t.interval) 
    df['prediction'] = pred
    return df

In [17]:
X_test_small = X_test.sample(frac=0.001, random_state=1)
print(X_test_small.shape)

(23014, 15)


In [18]:
%%time
#X_test_pred = compute_prediction(clfs, X_test_small)
X_test_pred = compute_prediction(clfs, X_test)
X_test_pred.head(20)

(23013803, 13)
Prediction took 1204.686 sec.

CPU times: user 15min 41s, sys: 4min 23s, total: 20min 5s
Wall time: 1min 28s


/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel/__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


### Metrics

In [19]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary(y_true, y_pred):
    m_acc = accuracy_score(y_true, y_pred)
    m_f1 = f1_score(y_true, y_pred)
    m_precision = precision_score(y_true, y_pred)
    m_recall = recall_score(y_true, y_pred)
    m_conf = confusion_matrix(y_true, y_pred)
    report = {'Accuracy':m_acc, 'Precision':m_precision, 'Recall':m_recall, 'F1':m_f1, 'Confusion Matrix':m_conf}
    return report

In [20]:
#https://github.com/miguelgfierro/codebase/blob/master/python/machine_learning/metrics.py
def classification_metrics_binary_prob(y_true, y_prob):
    m_auc = roc_auc_score(y_true, y_prob)
    m_logloss = log_loss(y_true, y_prob)
    report = {'AUC':m_auc, 'Log loss':m_logloss}
    return report

In [21]:
THRES = 0.5
threshold_func = lambda x: 0 if x <= THRES else 1


In [22]:
y_true = X_test_pred['ArrDelayBinary'].as_matrix()
y_prob = X_test_pred['prediction'].as_matrix()
y_pred = X_test_pred['prediction'].map(threshold_func).as_matrix()

In [23]:
report1 = classification_metrics_binary(y_true, y_pred)
report1

{'Accuracy': 0.73131428995025294,
 'Confusion Matrix': array([[9853828, 2239471],
        [3944009, 6976495]]),
 'F1': 0.69292135116035725,
 'Precision': 0.75700094813717844,
 'Recall': 0.63884368340508824}

In [24]:
report2 = classification_metrics_binary_prob(y_true, y_prob)
report2

{'AUC': 0.80508185334475957, 'Log loss': 0.53953094991838757}